# Simple Neural Network Implementation using Pytorch

Objectives :
1. Load dataset using Pytorch Dataset utilities package
2. Create the NN architecture using pytorch modules
3. Train the model on the dataset and report performance

In [26]:
import torch.nn as nn #Superclass module for nn
import torch.nn.functional as F #Contains functions that are used in the neural network
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, normalize

In [27]:
labels = pd.read_csv('/content/drive/MyDrive/Colab projects/Neural_Network_From_Scratch/data/Labels.csv')
data = pd.read_csv('/content/drive/MyDrive/Colab projects/Neural_Network_From_Scratch/data/Data.csv')

In [28]:
labels = labels.drop(columns=['Unnamed: 0'])
data = data.drop(columns=['Unnamed: 0'])

In [29]:
X_train = data.to_numpy()
X_train.shape

(13611, 16)

In [30]:
enc = OneHotEncoder()
ohe_labels = enc.fit_transform(labels)

In [31]:
y_train = ohe_labels.toarray()
y_train.shape

(13611, 7)

### Normalizing values since they are very large



In [32]:
X_train_norm = normalize(X_train)

In [33]:
mms = MinMaxScaler()
X_train_mms = mms.fit_transform(X_train)

In [34]:
X_train_norm

array([[7.03029742e-01, 1.51101505e-02, 5.15426686e-03, ...,
        7.79234997e-08, 2.06544515e-05, 2.47273322e-05],
       [7.01629942e-01, 1.55791930e-02, 4.89643631e-03, ...,
        8.70169590e-08, 2.22168288e-05, 2.43797806e-05],
       [7.03284783e-01, 1.49396551e-02, 5.09453297e-03, ...,
        7.29552158e-08, 1.97693069e-05, 2.39151808e-05],
       ...,
       [7.03435126e-01, 1.26755040e-02, 4.69980480e-03, ...,
        3.15213021e-08, 1.12993687e-05, 1.66392441e-05],
       [7.02684654e-01, 1.27339024e-02, 4.72462168e-03, ...,
        3.08773945e-08, 1.11409985e-05, 1.65925813e-05],
       [7.03342780e-01, 1.28833065e-02, 4.92389563e-03, ...,
        2.73571463e-08, 1.02804693e-05, 1.66527297e-05]])

### Create Dataset and DataLoader class and objects

In [35]:
class Data(Dataset):

  def __init__(self, X, Y):
    super().__init__()
    self.X = torch.Tensor(X)
    self.Y = torch.tensor(Y)

  def __getitem__(self, index):
    '''
    Returns the ith datapoint
    '''
    return self.X[index], self.Y[index]

  def __len__(self):
    '''
    Returns the length of the dataset
    '''
    return len(self.X)



In [36]:
dataset_train = Data(X_train_norm, y_train)

In [37]:
#Testing the len and getitem functions
print(len(dataset_train))
print(dataset_train[0])

13611
(tensor([7.0303e-01, 1.5110e-02, 5.1543e-03, 4.3053e-03, 2.9641e-05, 1.3613e-05,
        7.1095e-01, 4.7077e-03, 1.8914e-05, 2.4483e-05, 2.3720e-05, 2.2614e-05,
        1.8152e-07, 7.7924e-08, 2.0654e-05, 2.4727e-05]), tensor([0., 0., 0., 0., 0., 1., 0.], dtype=torch.float64))


In [38]:
#DataLoader is a pytorch iterable over the dataset

train_dataloader = DataLoader(dataset_train, shuffle=True, batch_size=128, num_workers=2, drop_last=True)

In [39]:
# next(iter(...)) catches the first batch of the data loader

input, label = next(iter(train_dataloader))

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [40]:
input.shape, label.shape

(torch.Size([128, 16]), torch.Size([128, 7]))

### Create the model architecture

In [41]:
class SimpleNeuralNetwork(nn.Module):

  def __init__(self, input_size, output_size, layers):
    super().__init__()
    self.nn_layers = []
    #First layer
    self.nn_layers.append(nn.Linear(input_size, layers[0], bias=True))
    self.nn_layers.append(nn.ReLU())

    #Intermediate Layers
    for idx in range(len(layers)-1):
      self.nn_layers.append(nn.Linear(layers[idx], layers[idx+1], bias=True))
      self.nn_layers.append(nn.ReLU())

    #Last Layer
    self.nn_layers.append(nn.Linear(layers[-1], output_size, bias=True))
    self.nn_layers.append(nn.Softmax())

    self.network = nn.Sequential(*self.nn_layers)

  def forward(self, x):
    output = self.network(x)
    return output




In [42]:
model = SimpleNeuralNetwork(len(X_train[0]), len(y_train[0]), layers = [32,64])

In [43]:
model

SimpleNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=7, bias=True)
    (5): Softmax(dim=None)
  )
)

In [44]:
for name, param in model.named_parameters():
  print(f"Parameter : {name} Shape : {param.shape}")

Parameter : network.0.weight Shape : torch.Size([32, 16])
Parameter : network.0.bias Shape : torch.Size([32])
Parameter : network.2.weight Shape : torch.Size([64, 32])
Parameter : network.2.bias Shape : torch.Size([64])
Parameter : network.4.weight Shape : torch.Size([7, 64])
Parameter : network.4.bias Shape : torch.Size([7])


In [45]:
#Continue and add the loss and optimizers
#Write the code and loop to run the epochs and perform backprop etc

In [46]:
#Check GPU availability

gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")

Is the GPU available? True


In [47]:
#device object which points to the GPU if present, else CPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device : {device}")

Device : cuda


In [60]:
#Loss module

loss_fn = nn.CrossEntropyLoss()

#Optimizer module

optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

## Model training

1. Push all data and model parameters to the device of our choice (GPU if available).
2. 5 steps of training : load a batch, obtain the predictions, calculate the loss, backpropagate, and update.


In [49]:
model.to(device)

SimpleNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=7, bias=True)
    (5): Softmax(dim=None)
  )
)

In [65]:
#Ensure to set model in training mode (model.train()), so that certain NN modules like BatchNorm and Dropout function accordingly (Dropout is not used during test time)

def train_model(model, data_loader_train, loss_fn, optimizer, epochs=100):
  model.train()

  for epoch in range(epochs):
    for data,labels in data_loader_train:

      data = data.to(device)
      labels = labels.to(device)

      #NOT model.forward()
      preds = model(data)
      #preds = preds.squeeze(dim=1)

      #Loss computation. (Just send predicted and true values)
      loss = loss_fn(preds, labels)

      #print(f"Epoch : {epoch} Loss : {loss.item()}")

      #Before backward propagation, ensure that all gradients are set to 0
      optimizer.zero_grad()

      #Compute the backward propagation to get the gradients
      loss.backward()

      #Compute the new parameter values based on gradients
      optimizer.step()


In [67]:
train_model(model, train_dataloader, loss_fn, optimizer, 100)

In [80]:
def test_model(model, data_loader):
  model.eval()
  total_count = 0
  correct_count = 0
  for data, label in data_loader:
    data = data.to(device)
    label = label.to(device)

    preds = model(data)

    correct_count += (preds.argmax(axis=1)==label.argmax(axis=1)).sum()
    total_count += data.shape[0]

  print(total_count)
  print(correct_count)
  return (correct_count/total_count)

In [81]:
test_model(model, train_dataloader).item()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


13568
tensor(3534, device='cuda:0')


0.2604658007621765